<a href="https://colab.research.google.com/github/sampathn2005/google-ai-studio-text-gen/blob/main/function_calling_math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Importing Necessary Libraries

In [1]:
import os
import google.generativeai as genai

## API Key Setup

In [2]:
# Configure Gemini API
GOOGLE_API_KEY = "AIzaSyA_YOH0-UZl34pNjrh_TPrWSLSmhqvgQGA123"  # Replace with your actual API key
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## Configuring Gemini API

In [3]:
def configure_gemini():
    """Configures the Gemini API."""
    if not GOOGLE_API_KEY or GOOGLE_API_KEY == "YOUR_GOOGLE_AI_STUDIO_API_KEY":
        print("Error: Please set your GOOGLE_API_KEY.")
        return False
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        return True
    except Exception as e:
        print(f"Error configuring Gemini API: {e}")
        return False

if not configure_gemini():
    exit()

## Defining the Model Name

In [4]:
MODEL_NAME = "gemini-1.5-pro-latest"

## Defining the Math Function

In [5]:
# Define the function to perform addition and subtraction
def calculate(operation: str, num1: float, num2: float) -> float:
    """Performs addition or subtraction."""
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    else:
        return "Invalid operation"

## Defining the Function Declaration for Gemini

In [6]:
# Define function declaration
math_function_declaration = genai.types.FunctionDeclaration(
    name="calculate",
    description="Performs addition or subtraction.",
    parameters={
        "type": "object",
        "properties": {
            "operation": {"type": "string", "description": "The operation to perform: 'add' or 'subtract'."},
            "num1": {"type": "number", "description": "The first number."},
            "num2": {"type": "number", "description": "The second number."}
        },
        "required": ["operation", "num1", "num2"]
    }
)


## Creating the Model with Function Support

In [7]:
# Create model with function support
model = genai.GenerativeModel(MODEL_NAME, tools=[math_function_declaration])
chat_session = model.start_chat()

## Main Interaction Loop

In [8]:
def main():
    """Main interaction loop."""
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Exiting...")
            break

        response = chat_session.send_message(user_input)

        try:
            # Check if function call exists
            function_call = response.candidates[0].content.parts[0].function_call
            function_name = function_call.name
            arguments = dict(function_call.args)

            print(f"DEBUG: Function '{function_name}' called with arguments {arguments}")

            if function_name == "calculate":
                result = calculate(**arguments)
                print(f"Function Output: {result}")
                continue

        except (AttributeError, TypeError) as e:
            print(f"Gemini: {response.text}")

In [ ]:
if __name__ == "__main__":
    main()

You: 1
Gemini: This is not a valid query. Please provide an operation and two numbers.

You: addition
Gemini: This is not a valid query. Please specify two numbers and the operation as "add".

You: add
Gemini: This is not a valid query. Please specify two numbers and the operation as "add".

You: calculate
Gemini: This is not a valid query. Please specify two numbers and the operation as "add" or "subtract".

You: add 2 10
DEBUG: Function 'calculate' called with arguments {'num2': 10.0, 'operation': 'add', 'num1': 2.0}
Function Output: 12.0
You: sub 10 2
DEBUG: Function 'calculate' called with arguments {'num2': 2.0, 'operation': 'subtract', 'num1': 10.0}
Function Output: 8.0
